In [2]:
!pip install jupyter-dash

  Using cached jupyter_dash-0.3.1-py3-none-any.whl (49 kB)
  Using cached ansi2html-1.6.0-py3-none-any.whl (14 kB)
  Using cached Flask-1.1.2-py2.py3-none-any.whl (94 kB)
Processing /home/wengong/.cache/pip/wheels/28/60/30/2f9f3884e6c75ea2588026586e6b1a6e95922d7369ce2c8dba/dash-1.17.0-py3-none-any.whl
  Using cached itsdangerous-1.1.0-py2.py3-none-any.whl (16 kB)
Processing /home/wengong/.cache/pip/wheels/9f/db/79/5701e1753b0e5fb306a7198056308f759963c63fa41515d807/dash_table-4.11.0-py3-none-any.whl
  Using cached future-0.18.2.tar.gz (829 kB)
Processing /home/wengong/.cache/pip/wheels/90/b9/27/c290d05058f55b0d4af28f0a8602017ffe13895291f163115b/dash_core_components-1.13.0-py3-none-any.whl
  Using cached Flask_Compress-1.8.0-py3-none-any.whl (7.2 kB)
Processing /home/wengong/.cache/pip/wheels/c8/a8/46/2900a0845e7992a317eca9ba18e55d47c03ccc4e80d6a37e73/dash_html_components-1.1.1-py3-none-any.whl
Processing /home/wengong/.cache/pip/wheels/71/17/b0/8bebc086e55a01cf4036902b6ca05fce3fb4c7fd48

In [3]:
import pandas as pd
import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go

import jupyter_dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = jupyter_dash.JupyterDash(__name__)

# ------------------------------------------------------------------------------
# Import and clean data (importing csv into pandas)
df = pd.read_csv("intro_bees.csv")

In [4]:
df.head()

,Program,Year,Period,State,ANSI,Affected by,Pct of Colonies Impacted,state_code
0,SURVEY,2019,JAN THRU MAR,Alabama,1,Disease,1.8,AL
1,SURVEY,2019,JAN THRU MAR,Alabama,1,Other,3.1,AL
2,SURVEY,2019,JAN THRU MAR,Alabama,1,Pesticides,0.3,AL
3,SURVEY,2019,JAN THRU MAR,Alabama,1,Pests_excl_Varroa,22.7,AL
4,SURVEY,2019,JAN THRU MAR,Alabama,1,Unknown,9.0,AL


In [5]:
df.shape

(4590, 8)

In [6]:
df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()
df.reset_index(inplace=True)
df[:5]

,State,ANSI,Affected by,Year,state_code,Pct of Colonies Impacted
0,Alabama,1,Disease,2015,AL,0.05
1,Alabama,1,Disease,2016,AL,1.20
2,Alabama,1,Disease,2017,AL,2.25
3,Alabama,1,Disease,2018,AL,1.30
4,Alabama,1,Disease,2019,AL,1.80


In [7]:
df.shape

(1350, 6)

In [8]:
# ------------------------------------------------------------------------------
# App layout
app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    dcc.Dropdown(
        id="slct_year",
        options=[
             {"label": "2015", "value": 2015},
             {"label": "2016", "value": 2016},
             {"label": "2017", "value": 2017},
             {"label": "2018", "value": 2018}
        ],
        multi=False,
        value=2015,
        style={'width': "40%"}
    ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='my_bee_map', figure={})

])


# ------------------------------------------------------------------------------
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure')],
    [Input(component_id='slct_year', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)

    dff = df.copy()
    dff = dff[dff["Year"] == option_slctd]
    dff = dff[dff["Affected by"] == "Varroa_mites"]

    # Plotly Express
    fig = px.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='state_code',
        scope="usa",
        color='Pct of Colonies Impacted',
        hover_data=['State', 'Pct of Colonies Impacted'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
        template='plotly_dark'
    )

    # Plotly Graph Objects (GO)
    # fig = go.Figure(
    #     data=[go.Choropleth(
    #         locationmode='USA-states',
    #         locations=dff['state_code'],
    #         z=dff["Pct of Colonies Impacted"].astype(float),
    #         colorscale='Reds',
    #     )]
    # )
    #
    # fig.update_layout(
    #     title_text="Bees Affected by Mites in the USA",
    #     title_xanchor="center",
    #     title_font=dict(size=24),
    #     title_x=0.5,
    #     geo=dict(scope='usa'),
    # )

    return container, fig


# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=True)

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/wengong/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/home/wengong/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/wengong/projects/streamlit_venv/lib/python3.7/site-packages/retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "/home/wengong/projects/streamlit_venv/lib/python3.7/site-packages/retrying.py", line 212, in call
    raise attempt.get()
  File "/home/wengong/projects/streamlit_venv/lib/python3.7/site-packages/retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/home/wengong/projects/streamlit_venv/lib/python3.7/site-packages/six.py", line 703, in reraise
    raise value
  File "/home/wengong/projects/streamlit_venv/lib/python3.7/site-packages/retrying.py", line 200, in call
    attempt = Att